<a href="https://colab.research.google.com/github/NadilaAmaliaaa/MACHINE_LEARNING/blob/main/Jobsheet%204%20-%20Klasifikasi/ML_JS4_Tugas2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
import cv2
import os
import matplotlib.image as mpimg
from pathlib import Path
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import random

In [86]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [87]:
# Image directories
train_dir = '/content/drive/MyDrive/ML/images/training/'
test_dir = '/content/drive/MyDrive/ML/images/test/'

# Load dataset

In [88]:
def load_dataset(img_dir):
    p = Path(img_dir)
    dirs = p.glob('*')

    img_list = []

    for dir in dirs:
        label = str(dir).split('/')[-1]
        for file in dir.glob('*.jpg'):
            img = mpimg.imread(file)

            if img is not None:
                img_list.append((img, label))

    return img_list

# Standarisasi dan Encoding

In [89]:
def standarized_input(image):
    # Resize to w: 1100, h: 600 using OpenCV
    std_img = cv2.resize(image, (1100, 600))
    return std_img

def label_encoder(label):
    # day as 1; night as 0
    return 1 if label == 'day' else 0

# Preprocessing data

In [90]:
def preprocess(img_list):
    std_img_list = []

    for item in img_list:
        image = item[0]
        label = item[1]

        # Standarize the image
        std_img = standarized_input(image)

        # Encode the label
        img_label = label_encoder(label)

        std_img_list.append((std_img, img_label))

    return std_img_list

# Fungsi untuk ekstraksi histogram dari gambar

In [91]:
def extract_histogram(image_path, bins=(8, 8, 8)):
    # Baca gambar
    image = cv2.imread(image_path)
    # Konversi gambar ke format HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Ekstraksi histogram dalam ruang warna HSV
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
    # Normalisasi histogram
    cv2.normalize(hist, hist)
    return hist.flatten()

# Fungsi untuk memuat data dan label

In [92]:
def load_data(directory):
    data = []
    labels = []

    for label in ["day", "night"]:
        path = os.path.join(directory, label)
        for img_file in os.listdir(path):
            img_path = os.path.join(path, img_file)
            hist = extract_histogram(img_path)
            data.append(hist)
            labels.append(0 if label == "day" else 1)

    return np.array(data), np.array(labels)

# Memuat data training dan test

In [93]:
train_data, train_labels = load_data(train_dir)
test_data, test_labels = load_data(test_dir)

# Klasifikasi menggunakan SVM

In [94]:
# Inisialisasi model SVM
model = svm.SVC(kernel='linear')

# Melatih model SVM
model.fit(train_data, train_labels)

# Prediksi data test
predictions = model.predict(test_data)

# Evaluasi model
accuracy = accuracy_score(test_labels, predictions)
report = classification_report(test_labels, predictions, target_names=["Day", "Night"])

# Menampilkan hasil evaluasi
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:")
print(report)

Accuracy: 97.50%
Classification Report:
              precision    recall  f1-score   support

         Day       0.99      0.96      0.97        80
       Night       0.96      0.99      0.98        80

    accuracy                           0.97       160
   macro avg       0.98      0.98      0.97       160
weighted avg       0.98      0.97      0.97       160

